In [1]:
# %pip install scikit-learn
# %pip install seaborn
# %pip install dateparser
%pip freeze

anyio @ file:///C:/ci_311/anyio_1676425491996/work/dist
appdirs==1.4.4
archspec @ file:///croot/archspec_1697725767277/work
argon2-cffi @ file:///opt/conda/conda-bld/argon2-cffi_1645000214183/work
argon2-cffi-bindings @ file:///C:/ci_311/argon2-cffi-bindings_1676424443321/work
asttokens @ file:///opt/conda/conda-bld/asttokens_1646925590279/work
async-lru @ file:///C:/b/abs_e0hjkvwwb5/croot/async-lru_1699554572212/work
attrs @ file:///C:/b/abs_35n0jusce8/croot/attrs_1695717880170/work
Babel @ file:///C:/ci_311/babel_1676427169844/work
backcall @ file:///home/ktietz/src/ci/backcall_1611930011877/work
beautifulsoup4 @ file:///C:/b/abs_0agyz1wsr4/croot/beautifulsoup4-split_1681493048687/work
bleach @ file:///opt/conda/conda-bld/bleach_1641577558959/work
boltons @ file:///C:/ci_311/boltons_1677729932371/work
Brotli @ file:///C:/ci_311/brotli-split_1676435766766/work
bs4==0.0.1
certifi @ file:///C:/b/abs_91u83siphd/croot/certifi_1700501720658/work/certifi
cffi @ file:///C:/b/abs_924gv1kxzj/c

In [2]:


import config as cfg
import datetime
import importlib
import json
import numpy as np
import os
import pandas as pd
import pickle
import plotly.graph_objects as go
import sklearn_util as sklu
import logging
import sklearn_util as sklu
import timing
import training
importlib.reload(sklu)
importlib.reload(cfg)
importlib.reload(training)

# Initialize logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

if not os.path.exists('logs'):
    os.makedirs('logs')
filename = f"logs/{__name__}_{datetime.datetime.now().strftime('%Y-%m-%d-%H')}.log"
if not os.path.exists(filename):
    with open(filename, 'wt') as f:
        f.write(f'*** {filename} ***\n')   
        
if len(logger.handlers) < 1:
    handler = logging.FileHandler(filename)
    formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)

config_path = "config"
# config_file = "debug.json"
# config_file = "debug_dual_forecast_out.json"
# config_file = "debug_dual_forecast_out_6_months.json"
config_file = "backtest_2021.json"
# config_file = "backtest_2022.json"
# config_file = "backtest_2023.json"
# config_file = "daily_run_backtest_2023.json"
# config_file = "daily_run.json"

try:
    conf = cfg.load_config(config_path, config_file)
except json.JSONDecodeError as e:
    logger.error(f"Error loading JSON file: {config_file}. {e}")
# conf

features_to_log = [
    
    'Market Cap',
    'baseline',
    'conversion',
    'spanA',
    'spanB',
    'lagging_span',
    'volume',
    'high',
    'low',
    'open',
    'close',
    'adjClose',
    'EPS_2_trailing_mean',
    'EPS_4_trailing_mean',
    'EPS_8_trailing_mean',
    
    'eps', 'freeCashFlow',
    'epsdiluted', 'weightedAverageShsOut', 'weightedAverageShsOutDil',
    'deferredTaxLiabilitiesNonCurrent', 'longTermDebt', 'minorityInterest',
    'netDebt', 'othertotalStockholdersEquity', 'propertyPlantEquipmentNet',
    'shortTermInvestments', 'totalAssets', 'totalDebt', 'totalEquity',
    'totalLiabilitiesAndStockholdersEquity',
    'totalLiabilitiesAndTotalEquity', 'totalNonCurrentAssets',
    'totalStockholdersEquity', 'debtRepayment',
    'salesMaturitiesOfInvestments', 'adjDividend'
]


2024-02-20 21:12:22,035 - config - INFO - loading config at path config\daily_run.json


In [3]:


pickle_file_name = os.path.join(conf.feature_path, "ichimoku_price_features_df.pkl")
logger.info(f"Loading data from pickle file: {pickle_file_name}")
with open(pickle_file_name, 'rb') as file:
    try:
        ichimoku_price_features_df = pickle.load(file)
    except Exception as e:
        logger.warning(f"Error loading data from pickle file: {pickle_file_name} {e}")
        pass
    
# ichimoku_price_features_df.describe()


2024-02-20 21:12:22,101 - __main__ - INFO - Loading data from pickle file: features\2024-02-20\ichimoku_price_features_df.pkl


In [4]:
# np.max(ichimoku_price_features_df.index.get_level_values('Date'))
ichimoku_price_features_df = ichimoku_price_features_df[
    (ichimoku_price_features_df.index.get_level_values(conf.date_col) >= conf.start_date_extended) &
    (ichimoku_price_features_df.index.get_level_values(conf.date_col) <= conf.end_date_extended)].sort_index() 
# ichimoku_price_features_df


In [5]:

import os
import training
importlib.reload(training)

if not os.path.exists(conf.feature_path):
    os.makedirs(conf.feature_path)

def prep_for_training(dfreg):
    
    programmatic_features = set()
    
    log_features = {f'Log{column}': np.log(np.maximum(dfreg[column], 1)) for column in features_to_log} 
    programmatic_features.update(log_features.keys())
    dfreg = dfreg.assign(**log_features)

    def generate_offset_features(df, column, offsets: list):
        
        offset_features = {f'{column}_{offset}_days_ago': df[column].shift(offset) for offset in offsets}
        programmatic_features.update(offset_features.keys())
        # df = df.assign(**offset_features)
        
        log_offset_features = {f'Log{column}_{offset}_days_ago': np.log(np.maximum(df[column].shift(offset), 0.001)) for offset in offsets}
        programmatic_features.update(log_offset_features.keys())
        offset_features.update(log_offset_features)
        # df = df.assign(**log_offset_features)
        
        days_pct_change_features = {f'{column}_{offset}_days_pct_change': (df[column] - df[column].shift(offset)) / np.maximum(df[column].shift(offset), 0.001) * 100.0 for offset in offsets}
        programmatic_features.update(days_pct_change_features.keys())
        offset_features.update(days_pct_change_features)
        # df = df.assign(**days_pct_change_features)
        
        days_change_features = {f'{column}_{offset}_days_change': (df[column] - df[column].shift(offset)) for offset in offsets}
        programmatic_features.update(days_change_features.keys())
        offset_features.update(days_change_features)
        # df = df.assign(**days_change_features)
        
        # get rolling stats via offset and window since previous offset
        log_rolling_mean_features = {f'Log{column}_{offset}_rolling_mean': np.log(np.maximum(df[column].dropna().shift(offset).rolling(window=(next_offset-offset)).mean(), 0.001)) for offset, next_offset in zip(offsets, offsets[1:])}
        programmatic_features.update(log_rolling_mean_features.keys())
        offset_features.update(log_rolling_mean_features)
        # df = df.assign(**log_rolling_mean_features)
        
        log_rolling_max_features = {f'Log{column}_{offset}_rolling_max': np.log(np.maximum(df[column].dropna().shift(offset).rolling(window=(next_offset-offset)).max(), 0.001))for offset, next_offset in zip(offsets, offsets[1:])}
        programmatic_features.update(log_rolling_max_features.keys())
        offset_features.update(log_rolling_max_features)
        # df = df.assign(**log_rolling_max_features)
        
        log_rolling_min_features = {f'Log{column}_{offset}_rolling_min': np.log(np.maximum(df[column].dropna().shift(offset).rolling(window=(next_offset-offset)).min(), 0.001))for offset, next_offset in zip(offsets, offsets[1:])}
        programmatic_features.update(log_rolling_min_features.keys())
        offset_features.update(log_rolling_min_features)
        
        df = df.assign(**offset_features)
        
        return df

    for column in conf.columns_for_delta:
        dfreg = generate_offset_features(dfreg, column, conf.day_offsets)
        
    dfreg = dfreg.ffill() 
    
    dfreg['HL_PCT'] = (dfreg['high'] - dfreg['low']) / np.maximum(dfreg['adjClose'], 0.001) * 100.0
    dfreg['PCT_Open_close_change'] = (dfreg['adjClose'] - dfreg['open']) / np.maximum(dfreg['open'], 0.001) * 100.0
    # dfreg['PCT_Change'] = (dfreg['adjClose'] - dfreg['adjClose'].shift(1)) / dfreg['adjClose'].shift(1) * 100.0
    
    dfreg['dayofweek'] = dfreg.index.get_level_values(conf.date_col).dayofweek
    dfreg['dayofmonth'] = dfreg.index.get_level_values(conf.date_col).day
    dfreg['dayofyear'] = dfreg.index.get_level_values(conf.date_col).dayofyear
    # dfreg['weekofyear'] = dfreg.index.get_level_values('Date').weekofyear
    dfreg['quarter'] = dfreg.index.get_level_values(conf.date_col).quarter
    dfreg['year'] = dfreg.index.get_level_values(conf.date_col).year
    dfreg['day'] = dfreg.index.get_level_values(conf.date_col).day
    dfreg['month'] = dfreg.index.get_level_values(conf.date_col).month
        
    return dfreg[list(set(conf.features + list(programmatic_features) + conf.columns_for_delta))]
    
pickle_file_name = os.path.join(conf.feature_path, f"grouped_training_days.pkl")

if not conf.force_recalc_full_set and os.path.exists(pickle_file_name):
    with open(pickle_file_name, 'rb') as file:
        grouped_training_prep_df = pickle.load(file)
else:
    max_date = np.max(ichimoku_price_features_df.index.get_level_values(conf.date_col))
    grouped_training_prep_df = training.fill_missing_dates(ichimoku_price_features_df, conf, max_date)
    grouped_training_prep_df = grouped_training_prep_df.groupby(by='symbol', group_keys=False).apply(prep_for_training)
    logger.info(f"Saving data to pickle file: {pickle_file_name}")
    with open(pickle_file_name, 'wb') as file:
        pickle.dump(grouped_training_prep_df, file)

ichimoku_price_features_df = None


In [6]:
# grouped_training_prep_df['LogadjClose_3_rolling_mean']

In [7]:
def drop_non_index_object_columns(df):
    object_columns = df.select_dtypes(include='object').columns
    index_columns = df.index.names
    non_index_object_columns = [col for col in object_columns if col not in index_columns]
    return df.drop(columns=non_index_object_columns)
grouped_training_prep_df = drop_non_index_object_columns(grouped_training_prep_df)
grouped_training_prep_df


,,Loghigh_14_rolling_max,gradingCompany_newGrade_Colliers_Securities,Loglow_7_rolling_min,Loglow_1_rolling_min,Logopen_2_rolling_mean,weightedAverageShsOut,LoglongTermDebt,close_21_days_change,open_1_days_ago,Logopen_21_rolling_mean,...,vwap_3_days_pct_change,volume_3_days_pct_change,gradingCompany_newGrade_Mainfirst,open_2_days_change,adjClose_1_days_pct_change,gradingCompany_newGrade_SpareBank,Logclose_1_rolling_max,Price to Book,open_2_days_ago,gradingCompany_newGrade_CSLA
symbol,date,,,,,,,,,,,,,,,,,,,,,
A,2020-12-04,NaN,1.0,NaN,NaN,NaN,3.080000e+08,21.549194,NaN,NaN,NaN,...,NaN,NaN,2.0,NaN,NaN,1.0,NaN,7.128307,NaN,4.0
AAL,2020-12-04,NaN,1.0,NaN,NaN,NaN,5.090490e+08,24.327649,NaN,NaN,NaN,...,NaN,NaN,999999.0,NaN,NaN,999999.0,NaN,-1.510203,NaN,4.0
AAPL,2020-12-04,NaN,999999.0,NaN,NaN,NaN,1.705762e+10,25.315016,NaN,NaN,NaN,...,NaN,NaN,999999.0,NaN,NaN,999999.0,NaN,31.322388,NaN,999999.0
ABBV,2020-12-04,NaN,999999.0,NaN,NaN,NaN,1.769000e+09,25.133418,NaN,NaN,NaN,...,NaN,NaN,999999.0,NaN,NaN,999999.0,NaN,10.867707,NaN,999999.0
ABR,2020-12-04,NaN,1.0,NaN,NaN,NaN,1.137664e+08,22.000442,NaN,NaN,NaN,...,NaN,NaN,999999.0,NaN,NaN,999999.0,NaN,0.972659,NaN,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,2024-02-20,4.887601,1.0,4.834216,4.900783,4.897840,2.812810e+08,23.134885,1.65,135.23,4.860676,...,0.419476,-21.515804,999999.0,-0.21,-0.445931,999999.0,4.901936,-4.600438,134.00,999999.0
ZBH,2024-02-20,4.828314,1.0,4.780215,4.820443,4.805332,2.089000e+08,22.305928,1.62,124.15,4.810650,...,2.358722,-20.864362,2.0,2.78,0.216502,1.0,4.825991,1.779526,122.16,4.0
ZBRA,2024-02-20,5.563562,1.0,5.455107,5.604717,5.622464,0.000000e+00,21.511269,22.84,282.41,5.518112,...,7.400063,-7.675684,999999.0,-6.71,-1.281493,999999.0,5.618442,0.000000,276.57,999999.0


In [8]:

import modelling_prep as mp
import importlib
importlib.reload(mp)

# target_training_filled_df.dropna(how='all').isna().mean().sort_index(ascending=False).to_csv('training_drop_na_all_na_mean.csv')
high_nan_columns = mp.get_high_nan_columns(grouped_training_prep_df[\
    ((grouped_training_prep_df.index.get_level_values(conf.date_col) >= conf.start_date) &
    (grouped_training_prep_df.index.get_level_values(conf.date_col) <= conf.end_date))
    ], threshold=0.20)
grouped_training_prep_df_low_nan_df = mp.remove_columns(grouped_training_prep_df, high_nan_columns)
grouped_training_prep_df = None
grouped_training_prep_df_low_nan_df



,,Loghigh_14_rolling_max,gradingCompany_newGrade_Colliers_Securities,Loglow_7_rolling_min,Loglow_1_rolling_min,Logopen_2_rolling_mean,weightedAverageShsOut,LoglongTermDebt,close_21_days_change,open_1_days_ago,Logopen_21_rolling_mean,...,vwap_3_days_pct_change,volume_3_days_pct_change,gradingCompany_newGrade_Mainfirst,open_2_days_change,adjClose_1_days_pct_change,gradingCompany_newGrade_SpareBank,Logclose_1_rolling_max,Price to Book,open_2_days_ago,gradingCompany_newGrade_CSLA
symbol,date,,,,,,,,,,,,,,,,,,,,,
A,2020-12-04,NaN,1.0,NaN,NaN,NaN,3.080000e+08,21.549194,NaN,NaN,NaN,...,NaN,NaN,2.0,NaN,NaN,1.0,NaN,7.128307,NaN,4.0
AAL,2020-12-04,NaN,1.0,NaN,NaN,NaN,5.090490e+08,24.327649,NaN,NaN,NaN,...,NaN,NaN,999999.0,NaN,NaN,999999.0,NaN,-1.510203,NaN,4.0
AAPL,2020-12-04,NaN,999999.0,NaN,NaN,NaN,1.705762e+10,25.315016,NaN,NaN,NaN,...,NaN,NaN,999999.0,NaN,NaN,999999.0,NaN,31.322388,NaN,999999.0
ABBV,2020-12-04,NaN,999999.0,NaN,NaN,NaN,1.769000e+09,25.133418,NaN,NaN,NaN,...,NaN,NaN,999999.0,NaN,NaN,999999.0,NaN,10.867707,NaN,999999.0
ABR,2020-12-04,NaN,1.0,NaN,NaN,NaN,1.137664e+08,22.000442,NaN,NaN,NaN,...,NaN,NaN,999999.0,NaN,NaN,999999.0,NaN,0.972659,NaN,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,2024-02-20,4.887601,1.0,4.834216,4.900783,4.897840,2.812810e+08,23.134885,1.65,135.23,4.860676,...,0.419476,-21.515804,999999.0,-0.21,-0.445931,999999.0,4.901936,-4.600438,134.00,999999.0
ZBH,2024-02-20,4.828314,1.0,4.780215,4.820443,4.805332,2.089000e+08,22.305928,1.62,124.15,4.810650,...,2.358722,-20.864362,2.0,2.78,0.216502,1.0,4.825991,1.779526,122.16,4.0
ZBRA,2024-02-20,5.563562,1.0,5.455107,5.604717,5.622464,0.000000e+00,21.511269,22.84,282.41,5.518112,...,7.400063,-7.675684,999999.0,-6.71,-1.281493,999999.0,5.618442,0.000000,276.57,999999.0


In [9]:
# get the columns that are nan after 2024-02-20 and the count of nans for each column after 2024-02-20
grouped_training_prep_df_low_nan_df[
    (grouped_training_prep_df_low_nan_df.index.get_level_values(conf.date_col) >= pd.to_datetime(datetime.datetime(2024, 2, 19))) &
    (grouped_training_prep_df_low_nan_df.isna().any(axis=1))
    ]

# grouped_training_prep_df_low_nan_df[grouped_training_prep_df_low_nan_df.index.get_level_values('date') >= pd.to_datetime(2024, 2, 20)].isna().count()

,,Loghigh_14_rolling_max,gradingCompany_newGrade_Colliers_Securities,Loglow_7_rolling_min,Loglow_1_rolling_min,Logopen_2_rolling_mean,weightedAverageShsOut,LoglongTermDebt,close_21_days_change,open_1_days_ago,Logopen_21_rolling_mean,...,vwap_3_days_pct_change,volume_3_days_pct_change,gradingCompany_newGrade_Mainfirst,open_2_days_change,adjClose_1_days_pct_change,gradingCompany_newGrade_SpareBank,Logclose_1_rolling_max,Price to Book,open_2_days_ago,gradingCompany_newGrade_CSLA
symbol,date,,,,,,,,,,,,,,,,,,,,,


In [10]:

# # import gc
# # gc.collect()
# from itertools import product

# df2 = grouped_training_prep_df_low_nan_df
# prefill_path = os.path.join(conf.feature_path, "prefill_features.csv")
# # df2.to_csv(prefill_path)
# symbols = df2.index.get_level_values('symbol').unique()
# dates = pd.date_range(
#     start=df2.index.get_level_values(conf.date_col).min(), 
#     end=df2.index.get_level_values(conf.date_col).max(), 
#     freq='D')
# # Create all combinations of symbols and dates
# combinations = list(product(symbols, dates))
# # Convert these combinations into a DataFrame
# df1 = pd.DataFrame(combinations, columns=['symbol', conf.date_col]).set_index(['symbol',conf.date_col])
# # df_new
# # df = pd.merge(df_new, df, how='left', left_index=True, right_index=True)

# # df1 = pd.read_csv("yourdata.csv")
# # df2 = pd.read_csv("yourdata2.csv")
# # df2_key = df2.Colname2

# # creating a empty bucket to save result
# df_result = pd.DataFrame(columns=(df2.columns.append(df1.columns)).unique())
# # deleting df2 to save memory
# # del(grouped_training_prep_df_low_nan_df)
# # del(df2)
# results_path = os.path.join(conf.feature_path, "date_fill_results.csv")
# df_result.to_csv(results_path) # ,index_label=False)

# # save data which only appear in df1 # sorry I was doing left join here. no need to run below two line.
# # df_result = df1[df1.Colname1.isin(df2.Colname2)!=True]
# # df_result.to_csv("df3.csv",index_label=False, mode="a")

# @timing.timer_func
# def preprocess(x):
#     # df2=pd.merge(df1, x, left_on = "Colname1", right_on = "Colname2")
#     x = x.set_index(['symbol',conf.date_col])
#     logger.info(f"Processing chunk: {x.index.get_level_values(conf.date_col).min()} - {x.index.get_level_values(conf.date_col).max()}")
#     df2 = pd.merge(df1[
#         (df1.index.get_level_values(conf.date_col) >= x.index.get_level_values(conf.date_col).min()) &
#         (df1.index.get_level_values(conf.date_col) <= x.index.get_level_values(conf.date_col).max())
#         ], 
#     x, how='left', left_index=True, right_index=True)
#     df2.to_csv(results_path,mode="a",header=False,index=False)

# reader = pd.read_csv(prefill_path, chunksize=100)

# [preprocess(r) for r in reader]
# df2


In [11]:
  

# for training_config in conf.training_configs:
import training
importlib.reload(training)
logger.info(f"Training models for {len(conf.training_configs)} configurations")
for training_config in conf.training_configs:   
    training.train_models(training_config, conf, grouped_training_prep_df_low_nan_df) # .dropna())

    

2024-02-20 21:12:34,065 - __main__ - INFO - Training models for 3 configurations
2024-02-20 21:12:34,066 - training - INFO - training_config: {'forecast_out': 7, 'forecast_mean_trailing_days': 3, 'forecast_root_col': 'LogadjClose', 'forecast_col': 'LogadjClose_3_rolling_mean', 'num_training_samples': 10000, 'classify_confidence_threshold_up': 0.75, 'classify_confidence_threshold_down': 0.7, 'classify_change_threshold_ratio_up': 0.003, 'classify_change_threshold_ratio_down': -0.003, 'parent_config': <config.Config object at 0x000001F2D49A0F90>}
2024-02-20 21:12:34,067 - training - INFO - df (input): (411070, 465)
2024-02-20 21:12:34,068 - training - INFO - filling missing dates {df.shape}
c:\src\market\training.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy

LogisticRegression(max_iter=2000, random_state=1)

2024-02-20 21:13:21,189 - config - INFO - Saving forecast actuals for regression model
2024-02-20 21:13:21,202 - config - INFO - Saving data to pickle file: results\2024-02-20\log_regression_LogadjClose_3_rolling_mean_10000_smpl_7_day_0_75_confUp_0_7_confDown_0_003_changeUp_-0_003_changeDown.pkl
2024-02-20 21:13:21,223 - sklearn_util - INFO -  negative_actuals: 11 neutral_actuals: 0 positive_actuals: 7 total_actuals: 18
 negative_forecast: 1 neutral_forecast: 15 positive_forecast: 2


2024-02-20 21:13:21,633 - sklearn_util - INFO -  negative_actuals: 20 neutral_actuals: 5 positive_actuals: 2 total_actuals: 27
 negative_forecast: 17 neutral_forecast: 9 positive_forecast: 1


2024-02-20 21:13:21,666 - sklearn_util - INFO -  negative_actuals: 1 neutral_actuals: 1 positive_actuals: 23 total_actuals: 25
 negative_forecast: 1 neutral_forecast: 18 positive_forecast: 6


...

2024-02-20 21:15:01,853 - sklearn_util - INFO -  negative_actuals: 0 neutral_actuals: 14 positive_actuals: 19 total_actuals: 33
 negative_forecast: 0 neutral_forecast: 10 positive_forecast: 23


2024-02-20 21:15:01,909 - sklearn_util - INFO -  negative_actuals: 0 neutral_actuals: 15 positive_actuals: 19 total_actuals: 34
 negative_forecast: 6 neutral_forecast: 2 positive_forecast: 26


2024-02-20 21:15:01,953 - sklearn_util - INFO -  negative_actuals: 0 neutral_actuals: 5 positive_actuals: 19 total_actuals: 24
 negative_forecast: 2 neutral_forecast: 10 positive_forecast: 12


2024-02-20 21:15:02,000 - sklearn_util - INFO -  negative_actuals: 13 neutral_actuals: 9 positive_actuals: 2 total_actuals: 24
 negative_forecast: 11 neutral_forecast: 7 positive_forecast: 6


2024-02-20 21:15:02,067 - sklearn_util - INFO -  negative_actuals: 3522 neutral_actuals: 2213 positive_actuals: 4459 total_actuals: 10194
 negative_forecast: 1913 neutral_forecast: 6012 positive_forecast: 2269
2024-02-20 21:15:02,075 - training - INFO - global: training start_date: 2021-01-01 00:00:00 holdout start date: 2023-12-31 00:00:00 end_date: 2024-02-20 21:12:22.089783 
 model_type: log_regression forecast target: LogadjClose_7_rolling_mean forecast_out: 21 forecast_mean_trailing_days: 7 day_offsets: [1, 2, 3, 7, 14, 21, 28] total_holdout_base:30 
 model_configs: ['7_days_out_3_MA_today_30_holdout.json', '14_days_out_7_MA_today_30_holdout.json', '21_days_out_7_MA_today_30_holdout.json'] num samples: 10000 change threshold:0.003/-0.003 confidence threshold: 0.75/0.7 
 metrics: 
 MAE: 0.86 MSE: 0.91 RMSE: 0.95 MAPE: 85.61 % MTE: 0.86
2024-02-20 21:15:02,076 - training - INFO - metrics_string: 
 MAE: 0.86 MSE: 0.91 RMSE: 0.95 MAPE: 85.61 % MTE: 0.86 metrics_df: (10194, 19)


In [12]:
# grouped_training_prep_df_low_nan_df['LogadjClose_3_rolling_mean']
# grouped_training_prep_df_low_nan_df['LogadjClose_14_rolling_mean']

In [13]:
# grouped_training_prep_df_low_nan_df.dropna()